In [105]:
import nltk
import matplotlib.pyplot as plt
import numpy
from nltk.corpus import twitter_samples
import random
import numpy as np
import math as m


In [22]:
# use predefined function ‘fileids()’ to see the content
twitter_samples.fileids()
# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [23]:
# print positive in greeen
print('\033[92m' + all_positive_tweets[random.randint(0,5000)])

# print negative in red
print('\033[91m' + all_negative_tweets[random.randint(0,5000)])

@mindwiped but tomorrow lmao hehe :D
@mustntgrumble I hadn't thought of an icepack. Big doses of pain killers on board. But it is feeling very very painful already :( *whimpers*


In [160]:

from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer   # module for tokenizing strings
import re
import string                              # for string operations

stop_words = stopwords.words('english')
punctuation = string.punctuation

tweets = all_positive_tweets + all_negative_tweets ## Concatenate the lists. 
labels = np.append(np.ones((len(all_positive_tweets),1)), np.zeros((len(all_negative_tweets),1)), axis = 0)

train_pos  = all_positive_tweets[:4000]
train_neg  = all_negative_tweets[:4000]
train_x = train_pos + train_neg 
train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)), axis=0).reshape(-1)


test_pos  = all_positive_tweets[1000:]
test_neg  = all_negative_tweets[1000:]
test_x = test_pos + test_neg
test_y = np.append(np.ones(len(test_pos)),np.zeros(len(test_neg)), axis=0)



In [162]:
print(train_y)

[1. 1. 1. ... 0. 0. 0.]


In [44]:
def preprocess(tweet) :
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'https?://[^\s\n\r]+', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    stemmer = PorterStemmer()
    tweet_tokens = tokenizer.tokenize(tweet)
    cleaned_stemmed_tweet = []
    for token in tweet_tokens:
        if token not in stop_words and token not in punctuation :
            cleaned_stemmed_tweet.append(stemmer.stem(token))
    return cleaned_stemmed_tweet




In [47]:
tweet_processed = preprocess(all_positive_tweets[1456])
print(tweet_processed)

['u', 'cant', 'chang', 'peopl', 'feel', 'u', 'dnt', 'tri', 'live', 'ur', 'life', 'happi', ':)']


In [86]:
positive_dictionary = {}
negative_dictionary = {}

for i in range(len(all_positive_tweets)) :
    pos_tokens = preprocess(all_positive_tweets[i])
    neg_tokens = preprocess(all_negative_tweets[i])
    for token in pos_tokens :
        if token in positive_dictionary :
            positive_dictionary[token]+=1
        else :
            positive_dictionary[token]=1
    for token in neg_tokens :
        if token in negative_dictionary :
            negative_dictionary[token]+=1
        else :
            negative_dictionary[token]=1

In [114]:
def sentiment(tweet) :
    if tweet in all_negative_tweets:
        return 0
    return 1

In [127]:
def Vectorize(tweets, bias) :
    freq_table = [[]]
    for tweet in tweets:    
        toks = preprocess(tweet)
        negative_score = 0
        positive_score = 0
        for tok in toks:
            if tok in negative_dictionary :
                negative_score+=negative_dictionary[tok]
            if tok in positive_dictionary:
                positive_score+=positive_dictionary[tok]
        if len(toks)>0:
            freq_table.append((np.array([bias,positive_score,negative_score]),sentiment(tweet)))
    freq_table= freq_table[1:]
    return freq_table




In [199]:
def sigmoid(x, w):
    sig = []
    for value in x:
        sig.append(int((1.0 / (1 + np.double(np.exp(-np.dot(value[0], w)))))>0.5))
    return np.array(sig)

In [201]:
from sklearn.metrics import log_loss
weight = []




In [207]:
def train(x_train,y_train,learning_rate,epochs) :
    x_train_processed = Vectorize(x_train,1)
    weight = np.zeros(3)
    for epoch in range(epochs+1):
        sig = sigmoid(x_train_processed,weight)
        loss = log_loss(sig, y_train, eps = 1e-15, normalize = True, sample_weight = None, labels = [0,1])
        gradient = np.dot(np.transpose([x[0] for x in x_train_processed]), (sig - y_train)) / len(y_train)
        weight -= learning_rate * gradient
        if epoch  == 10000:
            print(f'Epoch {epoch}, Loss: {loss}')
            print(weight)
        #print(loss)

In [208]:
train(train_x,train_y,0.01,10000)

In [204]:
print(weight)

[]
